## get wrong data

In [27]:
import os
import re
import json
import numpy as np
import pandas as pd

In [ ]:
data_dir = '/home/yuqihang/projects/CoLLM/collm-datasets/yelpdu'

In [ ]:
train_ = pd.read_pickle(os.path.join(data_dir, 'train_ood2.pkl'))

In [28]:
id2title = json.load(open(os.path.join(data_dir, 'id2title.json'), 'r'))

In [6]:
n_clusters = 4
mf_ = pd.read_csv(os.path.join(data_dir, f"mf_user_group_{n_clusters}.csv"))
lightgcn_ = pd.read_csv(os.path.join(data_dir, f"lightgcn_user_group_{n_clusters}.csv"))
sasrec_ = pd.read_csv(os.path.join(data_dir, f"sasrec_user_group_{n_clusters}.csv"))

In [8]:
lightgcn = lightgcn_[lightgcn_['cluster']==0]

In [59]:
sasrec = sasrec_[sasrec_['cluster']==2]

In [60]:
len(sasrec)

1594

In [13]:
lightgcn.head(2)

,user_id,cluster
7,8,0
19,20,0


In [22]:
train = train_[train_.uid.isin(list(lightgcn.user_id))].reset_index(drop=True)

In [42]:
train.head(2)

,uid,iid,label,timestamp,flag,not_cold,new_his,new_his_label
0,8,56004,0,1.279321e+09,-1.0,1,"[52117, 29932, 13421, 2950, 3433, 18779, 9905,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,8,55777,1,1.279322e+09,-1.0,1,"[29932, 13421, 2950, 3433, 18779, 9905, 29146,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"


In [30]:
train['length']=train['his'].apply(lambda x:len(x) if x[0]!=0 else len(x)-1)

In [46]:
train_.shape

(29678, 9)

In [ ]:
train = train[train.length>10]

In [40]:
train['new_his'] = train['his'].apply(lambda x: x[-10:])
train['new_his_label'] = train['his_label'].apply(lambda x: x[-10:])

In [41]:
del train['his']
del train['his_label']
del train['length']

In [43]:
train['sum'] = train['new_his_label'].apply(lambda x: sum(x))

In [ ]:
train = train[train['sum']==5].reset_index(drop=True)

In [61]:
sasrec.head(2)

,user_id,cluster,embedding
8,9,2,"[-0.006025682669132948, -1.1818451881408691, 1..."
18,19,2,"[-0.006059326697140932, -4.829181671142578, 6...."


In [64]:
train_ = train[train['uid'].isin(list(sasrec['user_id']))]

In [65]:
train_.shape

(808, 9)

In [67]:
train_ = train_.sample(n=300,random_state=2025)

In [71]:
train = train_.reset_index(drop=True)

In [72]:
train.head(2)

,uid,iid,label,timestamp,flag,not_cold,new_his,new_his_label,sum
0,4727,59036,1,1.317657e+09,-1.0,1,"[35792, 56519, 44856, 35299, 21383, 21209, 213...","[0, 1, 0, 1, 1, 0, 1, 0, 0, 1]",5
1,5745,56744,0,1.427460e+09,-1.0,1,"[35305, 2097, 46950, 9987, 17520, 43418, 28222...","[0, 0, 1, 1, 0, 0, 1, 1, 0, 1]",5


In [73]:
save_path = '/home/yuqihang/projects/CoLLM/collm-datasets/ml1mdu/reason/wrong_.txt'

In [76]:
for idx in range(len(train)):
    line = f'{train.uid[idx]}\sep{train.iid[idx]}\sep{list(train.new_his[idx])}\sep{train.label[idx]}\sep{list(train.new_his_label[idx])}\sep{convert_title_list_v4(turnid2txt(train.new_his[idx]),train.new_his_label[idx])}\sep{convert_title_list_v4(turnid2txt(train.iid[idx]))}'
    print(line,file=open(save_path,'a'))

In [57]:
txt = convert_title_list_v4(turnid2txt(train.new_his[0]),train.new_his_label[0])

In [ ]:
# data = line.split('\sep')
# uid.append(int(data[0]))
# iid.append(int(data[1]))
# his.append(eval(data[2]))
# label.append(int(data[3]))
# his_label.append(eval(data[4]))
# his_title.append(data[5].replace('with feature <ItemID> ',''))
# title.append(data[6].replace('\n',''))
# 5944\sep1073\sep[407, 905, 1507, 1529, 2293, 123, 1337, 754, 1076, 1089]\sep0\sep[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]\sep"Patriot, The" (evaluation: Yes) with description: Action, Drama, War; "Seven Years in Tibet" (evaluation: No) with description: Drama, War; "To Be or Not to Be" (evaluation: No) with description: Comedy, Drama, War; "English Patient, The" (evaluation: No) with description: Drama, Romance, War; "Heartbreak Ridge" (evaluation: No) with description: Action, War; "Thin Red Line, The" (evaluation: No) with description: Action, Drama, War; "Red Dawn" (evaluation: No) with description: Action, War; "G.I. Jane" (evaluation: No) with description: Action, Drama, War; "Mars Attacks!" (evaluation: No) with description: Action, Comedy, Sci-Fi, War; "Hot Shots! Part Deux" (evaluation: No) with description: Action, Comedy, War\sep"Rambo: First Blood Part II" with description: Action, War

In [25]:
def convert_str_2_list(x):
    return np.array([int(item) for item in re.split(r'\s+', x[1:-1].replace(',',' ').strip(' '))])
def dftype(df):
    for idx in df.columns:
        if isinstance(df[idx][0],list) or isinstance(df[idx][0],np.ndarray):
            print(idx,type(df[idx][0]),type(df[idx][0][0]))
        else:
            print(idx,type(df[idx][0]))

In [56]:
def turn2list(sample):
    if isinstance(sample,list):
        sample = sample
    elif isinstance(sample, str):
        sample = [sample]
    else:
        try:
            sample = list(sample)
        except:
            raise ValueError("sample must be a list or a str")
    return sample

def turnid2txt(ids):
    if isinstance(ids,(int,np.integer)):
        ids = [ids]
    elif isinstance(ids,list):
        pass
    else:
        try :
            ids = list(ids)
        except:
            raise NotImplementedError(f"Unsupported type of ids {type(ids)}")
    titles = [id2title[str(idx)]['title'] for idx in ids]
    descriptions = None
    descriptions = [id2title[str(idx)]['keywords'] for idx in ids]
    return titles, descriptions
    
def convert_title_list_v4(samples=None,labels=None,withid=False):
    if len(samples) == 2:
        titles,descs = samples
    elif len(samples) == 1:
        titles = samples
        desc = None
    else:
        raise ValueError("size of samples must be 1 or 2")

    titles = turn2list(titles)
    if labels is not None:
        label2eval = {0:"No",1:"Yes"}
        labels = [label2eval[label] for label in turn2list(labels)]
    if descs is not None:
        descs = turn2list(descs)
    
    itemsymbol = '<ItemID>'
    items = []

    for idx, title in enumerate(titles):
        if title.strip(" ")=="":
            item = f'"Unknow"'
        else:
            item = f'"{title.strip(" ")}"'
        if withid:
            item = f'{item} with feature {itemsymbol}'
        if labels is not None:
            item = f'{item} (evaluation: {labels[idx]})'
        if descs is not None:
            desc = descs[idx].strip(' ')
            desc = re.sub(r'<[^>]+>', '', desc)
            desc = re.sub(r'\s+', ' ', desc)
            item = f'{item} with description: {desc}'
        items.append(item)
    # print("Example of prompts:", items[0])

    if len(items)>0:
        return "; ".join(items)
    else:
        return "unkow"

In [77]:
dftype(train)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
timestamp <class 'numpy.float64'>
flag <class 'numpy.float64'>
not_cold <class 'numpy.int64'>
new_his <class 'numpy.ndarray'> <class 'numpy.int64'>
new_his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
sum <class 'numpy.int64'>


In [78]:
train.shape

(300, 9)

## load wrong data

In [15]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
wrong_file = '/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong_1.txt'

In [3]:
with open(wrong_file,'r') as f:
    wrong_list = f.readlines()
len(wrong_list)

In [5]:
uid = []
iid = []
his = [] 
label = []
his_label = []
his_title = []
title = []
for line in wrong_list:
    data = line.split('\sep')
    uid.append(int(data[0]))
    iid.append(int(data[1]))
    his.append(eval(data[2]))
    label.append(int(data[3]))
    his_label.append(eval(data[4]))
    his_title.append(data[5].replace('with feature <ItemID> ',''))
    title.append(data[6].replace('\n',''))

In [6]:
wrong_data = pd.DataFrame({"uid":uid,'iid':iid,'label':label, 'his':his,'his_label':his_label,'his_title':his_title,'title':title})

In [7]:
wrong_data.head()

,uid,iid,label,his,his_label,his_title,title
0,20927,40012,0,"[30252, 44599, 36606, 44751, 45031, 44960, 187...","[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]","""The Palisade (Lavender Shores)"" (evaluation: ...","""P.S. I Spook You"" with description: life, wri..."
1,8483,20626,1,"[40433, 20414, 20471, 33539, 39841, 34463, 347...","[1, 1, 0, 1, 0, 0, 0, 0, 1, 0]","""The Bad Boy Next Door"" (evaluation: Yes) with...","""The Bad Luck Bride: The Cavensham Heiresses"" ..."
2,6390,3886,1,"[20334, 10160, 9536, 37380, 15208, 19818]","[1, 0, 0, 1, 0, 1]","""Wintersong: A Novel"" (evaluation: Yes) with d...","""Greater Gotham: A History of New York City fr..."
3,5639,21757,1,"[40397, 42989, 36136, 2032, 23311, 38134, 2307...","[1, 0, 0, 1, 1, 1, 1, 1, 1, 1]","""Accidentally Compromising the Duke"" (evaluati...","""It Started with a Kiss (The Worthingtons)"" wi..."
4,15532,45963,0,"[33547, 33717, 33775, 43474, 40343, 33677, 169...","[1, 0, 1, 0, 1, 1, 0, 1, 1, 1]","""Trick (A Cocky Cage Fighter Novel) (Volume 7)...","""The Bartender (Modern Love) (Volume 1)"" with ..."


In [8]:
wrong_data['his_pad'] = wrong_data['his'].map(lambda x: [0]+x if len(x) < 10 else x)
wrong_data['his_label_pad'] = wrong_data['his_label'].map(lambda x: [0]+x if len(x) < 10 else x)

In [9]:
wrong_data = wrong_data[['uid','iid','label','his_pad','his_label_pad','his_title','title']]
wrong_data.columns = ['uid','iid','label','his','his_label','his_title','title']
wrong_data.head()

,uid,iid,label,his,his_label,his_title,title
0,20927,40012,0,"[30252, 44599, 36606, 44751, 45031, 44960, 187...","[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]","""The Palisade (Lavender Shores)"" (evaluation: ...","""P.S. I Spook You"" with description: life, wri..."
1,8483,20626,1,"[40433, 20414, 20471, 33539, 39841, 34463, 347...","[1, 1, 0, 1, 0, 0, 0, 0, 1, 0]","""The Bad Boy Next Door"" (evaluation: Yes) with...","""The Bad Luck Bride: The Cavensham Heiresses"" ..."
2,6390,3886,1,"[0, 20334, 10160, 9536, 37380, 15208, 19818]","[0, 1, 0, 0, 1, 0, 1]","""Wintersong: A Novel"" (evaluation: Yes) with d...","""Greater Gotham: A History of New York City fr..."
3,5639,21757,1,"[40397, 42989, 36136, 2032, 23311, 38134, 2307...","[1, 0, 0, 1, 1, 1, 1, 1, 1, 1]","""Accidentally Compromising the Duke"" (evaluati...","""It Started with a Kiss (The Worthingtons)"" wi..."
4,15532,45963,0,"[33547, 33717, 33775, 43474, 40343, 33677, 169...","[1, 0, 1, 0, 1, 1, 0, 1, 1, 1]","""Trick (A Cocky Cage Fighter Novel) (Volume 7)...","""The Bartender (Modern Love) (Volume 1)"" with ..."


In [10]:
wrong_data.to_csv(wrong_file.replace('.txt','.csv'),index=False)

## Get analysis

In [2]:
import os
import numpy as np
import pandas as pd

In [56]:
wrong_data = pd.read_csv('/home/yuqihang/projects/CoLLM/collm-datasets/bookdu/reflection/wrong_1.csv')

In [57]:
from tqdm import tqdm
import requests
from openai import OpenAI
url = 'https://api.siliconflow.cn/v1/'
api_key = 'sk-aqjpdmbxfmztkooxohnyxcbngysqhpgmjwxojihwwzuxojnj'

def get_response(client,message):
    for delay_secs in (2**x for x in range(0, 10)):
        try:
            response = client.chat.completions.create(
                model="Qwen/QwQ-32B",
                messages=message,
                stream=False, 
                max_tokens=1024
            )
            content = response.choices[0].message.content
            reasoning_content = response.choices[0].message.reasoning_content
            return content, reasoning_content
        except openai.OpenAIError as e:
            randomness_collision_avoidance = random.randint(0, 1000) / 1000.0
            sleep_dur = delay_secs + randomness_collision_avoidance
            print(f"Error: {e}. Retrying in {round(sleep_dur, 2)} seconds.")
            time.sleep(sleep_dur)
            continue

In [58]:
label2text = {0:'No',1:'Yes'}
prompt_template = '''A user has given high ratings to the following books: <HisItemList>. Using all available information, make a prediction about whether the user would enjoy the book titled <TargetItemTitle>?'''
hints = 'The correct response is <answer>. Reflect on multiple aspects based on historical information and reanalyze to determine if the target item will be liked by users. Explain the reason for the oversight based on the previous analysis.'

In [62]:
reasons = []
sbar = tqdm(total = len(wrong_data))
sbar.set_description('Generating Reasoning')
for idx in range(len(wrong_data)):
    row = wrong_data.iloc[idx]
    label = row['label']
    his_seq = row['his_title']
    target = row['title']
    message = prompt_template.replace('<HisItemList>',his_seq).replace('<TargetItemTitle>',target)
    his_message = [
        {"role": "system","content": "You are a helpful recommender answer with Yes or No."},
        {"role": "user","content": message}
    ]
    client = OpenAI(base_url=url,api_key=api_key)
    answer,reasoning_content = get_response(client,his_message)
    cnt = 1
    while label2text[label] not in answer:
        # his_message.pop(0)
        his_message.append({"role": "assistant","content": label2text[1-label]})
        his_message.append({"role": "user","content": hints.replace('<answer>',label2text[label])})
        answer,reasoning_content = get_response(client,his_message)
        cnt += 1
    reasons.append(f'{label2text[label]}. {reasoning_content}')
    sbar.set_postfix(cnt=cnt)
    if cnt > 5:
        print(f'cnt:{cnt}')
    sbar.update(1)
sbar.close()

Generating Reasoning:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Reasoning: 100%|██████████| 2/2 [01:14<00:00, 37.32s/it, cnt=2]


In [ ]:
wrong_data['reason'] = reasons

In [ ]:
wrong_data.to_csv(wrong_file.replace('.txt','.csv'),index=False)

## Merge csv

In [1]:
import os
import pandas as pd
import numpy as np
import re
from glob import glob

In [2]:
csv_dir = '/data/yuqihang/datasets/collm-datasets/yelpdu/reason'

In [6]:
csv_files = glob(f'{csv_dir}/*.csv')

In [7]:
csv_files

['/data/yuqihang/datasets/collm-datasets/yelpdu/reason/wrong__batch_6.csv',
 '/data/yuqihang/datasets/collm-datasets/yelpdu/reason/wrong__batch_1.csv',
 '/data/yuqihang/datasets/collm-datasets/yelpdu/reason/wrong__batch_3.csv',
 '/data/yuqihang/datasets/collm-datasets/yelpdu/reason/wrong__batch_5.csv',
 '/data/yuqihang/datasets/collm-datasets/yelpdu/reason/wrong__batch_4.csv',
 '/data/yuqihang/datasets/collm-datasets/yelpdu/reason/reason_.csv',
 '/data/yuqihang/datasets/collm-datasets/yelpdu/reason/wrong__batch_2.csv']

In [8]:
save_file = 'reason.csv'

In [9]:
dfs = []
for csv_file in csv_files:
    file_path = csv_file#os.path.join(csv_dir, csv_file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        if not dfs:
            dfs.append(df)
        else:
            if list(df.columns) == list(dfs[0].columns):
                dfs.append(df)
            else:
                print(f"表头不一致: {csv_file}")
    else:
        print(f"文件不存在: {csv_file}")
if dfs:
    df = pd.concat(dfs, ignore_index=True)
else:
    print("没有有效的文件可以合并")

In [10]:
def convert_str_2_list(x):
    return np.array([int(item) for item in re.split(r'\s+', x[1:-1].replace(',',' ').strip(' '))])
def dftype(df):
    for idx in df.columns:
        if isinstance(df[idx][0],list) or isinstance(df[idx][0],np.ndarray):
            print(idx,type(df[idx][0]),type(df[idx][0][0]))
        else:
            print(idx,type(df[idx][0]))

In [24]:
df['his_label'] = df['his_label'].map(lambda x: convert_str_2_list(x))
df['his'] = df['his'].map(lambda x: convert_str_2_list(x))

In [12]:
df['reason'] = df['reason'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))

In [13]:
dftype(df)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
his <class 'numpy.ndarray'> <class 'numpy.int64'>
his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
his_title <class 'str'>
title <class 'str'>
reason <class 'str'>


In [14]:
df.head()

,uid,iid,label,his,his_label,his_title,title,reason
0,30756,26802,1,"[8499, 12623, 48701, 20557, 8509, 42044, 52363...","[0, 1, 0, 1, 1, 0, 1, 0, 1, 0]","""Charlie Was a Sinner"" (evaluation: No) with d...","""Bardot Cafe"" with description: drinks, happy ...","Yes. Okay, let's try to figure out if the user..."
1,5264,46343,1,"[30869, 11569, 36587, 55412, 32091, 30197, 381...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]","""Tortas Frontera"" (evaluation: No) with descri...","""Joy Tsin Lau Chinese Restaurant"" with descrip...","Yes. Okay, so I need to figure out if the user..."
2,1439,52273,1,"[39738, 58332, 118, 21152, 25721, 45295, 59051...","[0, 1, 1, 0, 0, 0, 1, 1, 0, 1]","""Barcade"" (evaluation: No) with description: b...","""Wiljax Famous Sandwiches"" with description: l...","Yes. Okay, let's see. The user wants to know i..."
3,1439,53722,1,"[38161, 55432, 14865, 34957, 13335, 51403, 580...","[0, 1, 1, 0, 0, 0, 1, 0, 1, 1]","""The Farmhouse"" (evaluation: No) with descript...","""L'Oceano Ristorante"" with description: seafoo...","Yes. Alright, let's tackle this problem. The u..."
4,10706,9896,1,"[7248, 35554, 39957, 12701, 33502, 19315, 5481...","[0, 1, 0, 1, 0, 1, 1, 1, 0, 0]","""Drago's Seafood Restaurant"" (evaluation: No) ...","""Le Bercail Restaurant"" with description: kids...","Yes. Okay, let me try to work through this ste..."


In [15]:
if len(save_file) == 0:
    save_ids = [csv_file.split('.')[0].split('_')[1:] for csv_file in csv_files]
    save_path = f'{csv_dir}/wrong_{"_".join(sorted(sum(save_ids,[])))}.csv'
else:
    save_path = f'{csv_dir}/{save_file}'
if os.path.exists(save_path):
    print(f'{save_path} already exists, skip')
else:
    df.to_csv(save_path,index=False)
save_path,len(df)

('/data/yuqihang/datasets/collm-datasets/yelpdu/reason/reason.csv', 3800)

In [16]:
len(df),dftype(df)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
his <class 'numpy.ndarray'> <class 'numpy.int64'>
his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
his_title <class 'str'>
title <class 'str'>
reason <class 'str'>


(3800, None)

## Convert to training data

In [37]:
import re
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('/data/yuqihang/datasets/collm-datasets/yelpdu/reason/reason.csv')

In [25]:
df.head(2)

,uid,iid,label,his,his_label,his_title,title,reason,not_cold
0,30756,26802,1,"[8499, 12623, 48701, 20557, 8509, 42044, 52363...","[0, 1, 0, 1, 1, 0, 1, 0, 1, 0]","""Charlie Was a Sinner"" (evaluation: No) with d...","""Bardot Cafe"" with description: drinks, happy ...","Yes. Okay, let's try to figure out if the user...",1
1,5264,46343,1,"[30869, 11569, 36587, 55412, 32091, 30197, 381...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]","""Tortas Frontera"" (evaluation: No) with descri...","""Joy Tsin Lau Chinese Restaurant"" with descrip...","Yes. Okay, so I need to figure out if the user...",1


In [26]:
len(df)

3800

In [27]:
df['not_cold'] = pd.DataFrame(np.ones(df.shape[0]),index=df.index).astype("int")

In [28]:
wrong = df[['uid','iid','label','his','his_label','not_cold','reason']]

In [29]:
wrong.head(2)

,uid,iid,label,his,his_label,not_cold,reason
0,30756,26802,1,"[8499, 12623, 48701, 20557, 8509, 42044, 52363...","[0, 1, 0, 1, 1, 0, 1, 0, 1, 0]",1,"Yes. Okay, let's try to figure out if the user..."
1,5264,46343,1,"[30869, 11569, 36587, 55412, 32091, 30197, 381...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]",1,"Yes. Okay, so I need to figure out if the user..."


In [30]:
dftype(wrong)

uid <class 'numpy.int64'>
iid <class 'numpy.int64'>
label <class 'numpy.int64'>
his <class 'numpy.ndarray'> <class 'numpy.int64'>
his_label <class 'numpy.ndarray'> <class 'numpy.int64'>
not_cold <class 'numpy.int64'>
reason <class 'str'>


In [31]:
len(wrong)

3800

In [32]:
wrong.to_pickle('/data/yuqihang/datasets/collm-datasets/yelpdu/reason_ood2.pkl')